In [1]:
import torch
from transformers import BertTokenizer, BertModel

import logging

import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd


## 데이터 불러오기 및 캡션데이터 추출하기

In [2]:
data_type = 'train2017'
csv_filename = 'pro_cap_{}.csv'.format(data_type)
data = pd.read_csv(csv_filename)

print(len(data))
data.head()

586646


,Unnamed: 0,image_id,file_name,caption,length
0,770337,391895,000000391895.jpg,a man with a red helmet on a small moped on a ...,14
1,771687,391895,000000391895.jpg,man riding a motor bike on a dirt road on the ...,12
2,772707,391895,000000391895.jpg,a man riding on the back of a motorcycle,9
3,776154,391895,000000391895.jpg,a dirt path with a young person on a motor bik...,28
4,781998,391895,000000391895.jpg,a man in a red shirt and a red hat is on a mot...,18


In [3]:
caption = data.caption.to_list()
max_length = max(data.length)
print(max_length)

49


## bert 모델 사용하기

https://github.com/google-research/bert

위 링크 본문에 따르면 bert-base-uncased는 소문자 변환과 accent marks 제거가 된 데이터를
12 Layer, 768 hidden, 12 heads, 110M parameters
구조로 사전 학습 시켜둔 모델이다.

In [4]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased", output_hidden_states = True).to(device)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

### 문장 하나로 테스트

encode_plus : 문장 토큰화 (Token ID, Mask ID, Segment ID)
add_special_token : [CLS], [SEP] 토큰 자동 추가 옵션
padding : pad 추가 여부 및 방법
max_length : 최대 길이 지정

< result >
input_ids : 토큰 ids
token_type_ids : 첫번째 문장의 요소는 0, 두번째 문장의 요소는 1
attention_mask : 어떤 단어에 집중해야하는지. 만약 padding이 있다면 해당 위치는 0으로 표현됨.

In [5]:
test_sen = "my dog likes playing"
tokenizer.encode_plus(test_sen, add_special_token = True, padding = 'max_length', max_length = 51)

Keyword arguments {'add_special_token': True} not recognized.


{'input_ids': [101, 2026, 3899, 7777, 2652, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [6]:
test_sen = "my dog likes playing"
token_ids = tokenizer.encode_plus(test_sen, add_special_token=True, padding = 'max_length', max_length = 51)['input_ids']
attention_mask = tokenizer.encode_plus(test_sen, add_special_token=True, padding = 'max_length', max_length = 51)['attention_mask']

# 리스트 형태로 반환
token_list = tokenizer.convert_ids_to_tokens(token_ids)
# str 형태로 반환
token_str = tokenizer.decode(token_ids)

Keyword arguments {'add_special_token': True} not recognized.
Keyword arguments {'add_special_token': True} not recognized.


### tensor 변환 및 모델에 적용

Bert 모델은

input으로
token_id, token_type_ids(문장이 두 개라면), attention_mask를 필요로 하고,

output으로
1. last_hidden_state(마지막 layer의 hidden state)
2. pooler_output(첫번재 토큰 [CLS]의 마지막 hidden state)
3. hidden_states(모델의 각 layer의 hidden state 값들)을 출력한다.

In [7]:
tensor_token_ids = torch.tensor(token_ids).unsqueeze(0).to(device=device)
tensor_attention_mask = torch.tensor(attention_mask).unsqueeze(0).to(device=device)

print(tensor_token_ids)
print(tensor_attention_mask)

tensor([[ 101, 2026, 3899, 7777, 2652,  102,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0]], device='mps:0')
tensor([[1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0]], device='mps:0')


In [8]:
# last_hidden_state, pooler_output, hidden_states 순서로 이루어져 있음.
output = model(tensor_token_ids, attention_mask = tensor_attention_mask)

In [9]:
print(output.last_hidden_state.shape)
print(output.pooler_output.shape)
print(len(output.hidden_states))
print(output.hidden_states[0].shape)

torch.Size([1, 51, 768])
torch.Size([1, 768])
13
torch.Size([1, 51, 768])


In [10]:
print("BERT가 이해하고 있는 token(단어) 개수, hidden_size :", model.embeddings.word_embeddings)
print("한번에 받을 수 있는 최대 token 수, hidden_size(embedding의 차원) :", model.embeddings.position_embeddings)
print("한번에 받을 수 있는 최대 문장의 수, hidden_size(embedding의 차원) :", model.embeddings.token_type_embeddings)

BERT가 이해하고 있는 token(단어) 개수, hidden_size : Embedding(30522, 768, padding_idx=0)
한번에 받을 수 있는 최대 token 수, hidden_size(embedding의 차원) : Embedding(512, 768)
한번에 받을 수 있는 최대 문장의 수, hidden_size(embedding의 차원) : Embedding(2, 768)
